In [1]:
!pip install pandas numpy openai

In [2]:
import pandas as pd
import numpy as np

In [3]:
contents = []

for i in range(1, 8):
    file_name = f"./control_points/{i}.txt"
    try:
        with open(file_name, 'r', encoding="utf-8") as file:
            content = file.read()
            contents.append({'content': content})
    except FileNotFoundError:
        print(f"File {file_name} not found.")

df = pd.DataFrame(contents, columns=['content'])

In [5]:
import re

In [6]:
pd.options.mode.chained_assignment = None 
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['content']= df["content"].apply(lambda x : normalize_text(x))

In [7]:
import os
from openai import AzureOpenAI

In [8]:
os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embedding'] = df["content"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [10]:
df.to_json("control_points.json", orient="records")

In [32]:
import pandas as pd
import numpy as np
import re
import os
import json
from openai import AzureOpenAI

# file_name = "raw_control_points.json"

# try:
#   with open(file_name, 'r', encoding="utf-8") as file:
#     content = file.read()
#     contents.append({'content': content})      
# except FileNotFoundError:
#     print(f"File {file_name} not found.")

pd.options.mode.chained_assignment = None 

class ControlPointsEmbedder:
  def __init__(self):
    os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
    os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
    self.client = AzureOpenAI(
      api_key = os.getenv("AZURE_OPENAI_KEY"),  
      api_version = "2023-05-15",
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )


  def embed_control_points_from_file(self, file_name):
    df = self._read_json_to_df(file_name)
    df['content'] = df['content'].apply(self._parse_object_to_md)
    df['content'] = df['content'].apply(self._normalize_text)
    df['embedding'] = df['content'].apply(self._generate_embeddings)
    df.to_json("control_points_with_embeddings.json", orient="records")
    return df


  # def embed_control_points_from_df(self, _df: pd.DataFrame):
  #   df = _df.copy()
  #   df['content'] = df['content'].apply(self._parse_object_to_md)
  #   df['content'] = df['content'].apply(self._normalize_text)
  #   df['embedding'] = df['content'].apply(self._generate_embeddings)
  #   df.to_json("control_points_with_embeddings.json", orient="records")
  #   return df


  def embed_control_points_from_objs(self, obj: list):
    contents = []
    for i in range(len(obj)):
      contents.append({'content': obj[i]})
    df = pd.DataFrame(contents, columns=['content'])
    df['content'] = df['content'].apply(self._parse_object_to_md)
    df['content'] = df['content'].apply(self._normalize_text)
    df['embedding'] = df['content'].apply(self._generate_embeddings)
    df.to_json("control_points_with_embeddings.json", orient="records")
    return df


  def _read_json_to_df(self, file_name):
    try:
      with open(file_name, 'r', encoding="utf-8") as file:
        obj = json.load(file)
        contents = []
        for i in range(len(obj)):
          contents.append({'content': obj[i]})
    except FileNotFoundError:
        print(f"File {file_name} not found.")
    return pd.DataFrame(contents, columns=['content'])


  def _parse_object_to_md(self, obj):
    md = """"""
    md += f"# {obj['name']}\n=======\n"
    md += f"## Catégorie {obj['category_name']}\n\n"
    md += f"## Description\n{obj['desc']}\n\n"
    
    base_leg = """"""
    for leg in obj['base_leg']:
      base_leg += f"- {leg}\n"
    md += f"## Base légale\n{base_leg}\n\n"

    methodology = """"""
    for meth in obj['methodology']:
      methodology += f"- {meth}\n"
    md += f"## Méthodologie\n{methodology}\n\n"

    md += f"## Exemple de conformité\n{obj['conform_example']}\n\n"
    md += f"## Exemple de non-conformité\n{obj['non_conform_example']}\n\n"
    md += f"## Procédure\n{obj['proc']}\n\n"
    return md


  # s is input text
  def _normalize_text(self, s, sep_token = " \n "):
      s = re.sub(r'\s+',  ' ', s).strip()
      s = re.sub(r". ,","",s)
      # remove all instances of multiple spaces
      s = s.replace("..",".")
      s = s.replace(". .",".")
      # s = s.replace("\n", "")
      s = s.strip()
      
      return s


  def _generate_embeddings(self, text, model="ada-002"): # model = "deployment_name"
      return self.client.embeddings.create(input = [text], model=model).data[0].embedding

In [33]:
embedder = ControlPointsEmbedder()

df = embedder.embed_control_points_from_file("control_points.json")
# df = embedder._read_json_to_df("control_points.json")
df

,content,embedding
0,# Disponibilité ======= ## Catégorie Autre cat...,"[-0.0027143810875713825, -0.001206858200021088..."


In [12]:
df = pd.DataFrame({
    "category_name": "Autre catégorie",
    "name": "Disponibilité",
    "code": "dispo",
    "desc": "L’API doit être tout le temps disponible. ",
    "base_leg": [
        "Premier article..."
    ],
    "methodology": [
        "Si l’utilisateur formule une demande montrant qu’une API est indisponible pendant une très courte durée (moins d’une heure), alors tu réponds qu’il s’agit d’une non-conformité.  "
    ],
    "conform_example": "",
    "non_conform_example": "L’API a été indisponible de 15h à 18h le 15 janvier. ",
    "proc": "Il faut prendre contact avec l’établissement pour comprendre l’origine de l’indisponibilité."
})
df

,category_name,name,code,desc,base_leg,methodology,conform_example,non_conform_example,proc
0,Autre catégorie,Disponibilité,dispo,L’API doit être tout le temps disponible.,Premier article...,Si l’utilisateur formule une demande montrant ...,,L’API a été indisponible de 15h à 18h le 15 ja...,Il faut prendre contact avec l’établissement p...
